In [1]:
import cv2
import numpy as np
import os

# Load HAAR face classifier
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def face_extractor(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)

    if len(faces) == 0:
        return None

    for (x, y, w, h) in faces:
        cropped_face = img[y:y+h, x:x+w]

    return cropped_face

def collect_face_samples(name, data_path='D://Projects//Face-Detection-using-OpenCV//image_samples'):
    cap = cv2.VideoCapture(0)
    count = 0

    while True:
        ret, frame = cap.read()
        detected_face = face_extractor(frame)

        if detected_face is not None:
            count += 1
            face = cv2.resize(detected_face, (200, 200))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

            file_name_path = os.path.join(data_path, f'{name}_image{count}.jpg')
            cv2.imwrite(file_name_path, face)

            cv2.putText(face, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
            cv2.imshow('Face Cropper', face)
        else:
            print("Face not found")
            pass

        if cv2.waitKey(1) == 13 or count == 100:
            break

    cap.release()
    cv2.destroyAllWindows()
    print("Collecting Samples Complete")

def train_face_recognition(data_path='D://Projects//Face-Detection-using-OpenCV//image_samples'):
    onlyfiles = [f for f in os.listdir(data_path) if os.path.isfile(os.path.join(data_path, f))]

    Training_Data, Labels = [], []

    for i, file in enumerate(onlyfiles):
        image_path = os.path.join(data_path, file)
        print("Loading image:", image_path)
        images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        
        if images is not None:
            Training_Data.append(np.asarray(images, dtype=np.uint8))
            Labels.append(i)
            print("Image loaded successfully.")
        else:
            print("Error loading image:", image_path)

    print("Number of training images loaded:", len(Training_Data))

    Labels = np.asarray(Labels, dtype=np.int32)
    model = cv2.face.LBPHFaceRecognizer_create() 

    model.train(np.asarray(Training_Data), np.asarray(Labels))
    print("Model trained successfully")

    return model

def recognize_face(model, name):
    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()
        image, face = face_detector(frame)

        try:
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

            results = model.predict(face)
            print(results)
            if results[1] < 500:
                confidence = int(100 * (1 - (results[1])/400))
                display_string = f'{confidence}% Confident it is {name}'

            cv2.putText(image, display_string, (100, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 120, 150), 2)

            if confidence > 80:
                cv2.putText(image, f'Hey {name}', (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
                cv2.imshow('Face Recognition', image )
            else:
                cv2.putText(image, "I don't know", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 2)
                cv2.imshow('Face Recognition', image )

        except Exception as e:
            print(f"Error Recognising Face: {e}")
            cv2.putText(image, "Error Recognising Face", (130, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 2)
            cv2.putText(image, "Locked", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 2)
            cv2.imshow('Face Recognition', image )
            pass

        if cv2.waitKey(1) == 13:
            break

    cap.release()
    cv2.destroyAllWindows()

def face_detector(img, size=0.5):
    # Convert image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    
    if len(faces) == 0:
        return img, []

    for (x, y, w, h) in faces:
        cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 255), 2)
        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi, (200, 200))
        
    return img, roi

if __name__ == "__main__":
    # Get user's name
    Name = input("Enter Your name here:")

    # Step 1: Collect face samples
    collect_face_samples(Name)

    # Step 2: Train face recognition model
    model = train_face_recognition()

    # Step 3: Recognize face using the trained model
    recognize_face(model, Name)


Enter Your name here:Samyak
Face not found
Face not found
Face not found
Loading image: D://Projects//Face-Detection-using-OpenCV//image_samples\Samyak_image1.jpg
Image loaded successfully.
Loading image: D://Projects//Face-Detection-using-OpenCV//image_samples\Samyak_image10.jpg
Image loaded successfully.
Loading image: D://Projects//Face-Detection-using-OpenCV//image_samples\Samyak_image100.jpg
Image loaded successfully.
Loading image: D://Projects//Face-Detection-using-OpenCV//image_samples\Samyak_image11.jpg
Image loaded successfully.
Loading image: D://Projects//Face-Detection-using-OpenCV//image_samples\Samyak_image12.jpg
Image loaded successfully.
Loading image: D://Projects//Face-Detection-using-OpenCV//image_samples\Samyak_image13.jpg
Image loaded successfully.
Loading image: D://Projects//Face-Detection-using-OpenCV//image_samples\Samyak_image14.jpg
Image loaded successfully.
Loading image: D://Projects//Face-Detection-using-OpenCV//image_samples\Samyak_image15.jpg
Image load

Image loaded successfully.
Loading image: D://Projects//Face-Detection-using-OpenCV//image_samples\Samyak_image89.jpg
Image loaded successfully.
Loading image: D://Projects//Face-Detection-using-OpenCV//image_samples\Samyak_image9.jpg
Image loaded successfully.
Loading image: D://Projects//Face-Detection-using-OpenCV//image_samples\Samyak_image90.jpg
Image loaded successfully.
Loading image: D://Projects//Face-Detection-using-OpenCV//image_samples\Samyak_image91.jpg
Image loaded successfully.
Loading image: D://Projects//Face-Detection-using-OpenCV//image_samples\Samyak_image92.jpg
Image loaded successfully.
Loading image: D://Projects//Face-Detection-using-OpenCV//image_samples\Samyak_image93.jpg
Image loaded successfully.
Loading image: D://Projects//Face-Detection-using-OpenCV//image_samples\Samyak_image94.jpg
Image loaded successfully.
Loading image: D://Projects//Face-Detection-using-OpenCV//image_samples\Samyak_image95.jpg
Image loaded successfully.
Loading image: D://Projects//F

Error Recognising Face: OpenCV(4.8.0) :-1: error: (-5:Bad argument) in function 'cvtColor'
> Overload resolution failed:
>  - src is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'src'

Error Recognising Face: OpenCV(4.8.0) :-1: error: (-5:Bad argument) in function 'cvtColor'
> Overload resolution failed:
>  - src is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'src'

Error Recognising Face: OpenCV(4.8.0) :-1: error: (-5:Bad argument) in function 'cvtColor'
> Overload resolution failed:
>  - src is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'src'

Error Recognising Face: OpenCV(4.8.0) :-1: error: (-5:Bad argument) in function 'cvtColor'
> Overload resolution failed:
>  - src is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'src'

Error Recognising Face: OpenCV(4.8.0) :-1: error: (-5:Bad argument) in function 'cvtColor'
> Overload resolution failed:
>  - src is

Error Recognising Face: OpenCV(4.8.0) :-1: error: (-5:Bad argument) in function 'cvtColor'
> Overload resolution failed:
>  - src is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'src'

Error Recognising Face: OpenCV(4.8.0) :-1: error: (-5:Bad argument) in function 'cvtColor'
> Overload resolution failed:
>  - src is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'src'

Error Recognising Face: OpenCV(4.8.0) :-1: error: (-5:Bad argument) in function 'cvtColor'
> Overload resolution failed:
>  - src is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'src'

Error Recognising Face: OpenCV(4.8.0) :-1: error: (-5:Bad argument) in function 'cvtColor'
> Overload resolution failed:
>  - src is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'src'

Error Recognising Face: OpenCV(4.8.0) :-1: error: (-5:Bad argument) in function 'cvtColor'
> Overload resolution failed:
>  - src is

Error Recognising Face: OpenCV(4.8.0) :-1: error: (-5:Bad argument) in function 'cvtColor'
> Overload resolution failed:
>  - src is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'src'

Error Recognising Face: OpenCV(4.8.0) :-1: error: (-5:Bad argument) in function 'cvtColor'
> Overload resolution failed:
>  - src is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'src'

(77, 42.77647259152901)
Error Recognising Face: OpenCV(4.8.0) :-1: error: (-5:Bad argument) in function 'cvtColor'
> Overload resolution failed:
>  - src is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'src'

Error Recognising Face: OpenCV(4.8.0) :-1: error: (-5:Bad argument) in function 'cvtColor'
> Overload resolution failed:
>  - src is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'src'

Error Recognising Face: OpenCV(4.8.0) :-1: error: (-5:Bad argument) in function 'cvtColor'
> Overload resolu

...

Error Recognising Face: OpenCV(4.8.0) :-1: error: (-5:Bad argument) in function 'cvtColor'
> Overload resolution failed:
>  - src is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'src'

Error Recognising Face: OpenCV(4.8.0) :-1: error: (-5:Bad argument) in function 'cvtColor'
> Overload resolution failed:
>  - src is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'src'

Error Recognising Face: OpenCV(4.8.0) :-1: error: (-5:Bad argument) in function 'cvtColor'
> Overload resolution failed:
>  - src is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'src'

Error Recognising Face: OpenCV(4.8.0) :-1: error: (-5:Bad argument) in function 'cvtColor'
> Overload resolution failed:
>  - src is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'src'

Error Recognising Face: OpenCV(4.8.0) :-1: error: (-5:Bad argument) in function 'cvtColor'
> Overload resolution failed:
>  - src is

Error Recognising Face: OpenCV(4.8.0) :-1: error: (-5:Bad argument) in function 'cvtColor'
> Overload resolution failed:
>  - src is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'src'

Error Recognising Face: OpenCV(4.8.0) :-1: error: (-5:Bad argument) in function 'cvtColor'
> Overload resolution failed:
>  - src is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'src'

Error Recognising Face: OpenCV(4.8.0) :-1: error: (-5:Bad argument) in function 'cvtColor'
> Overload resolution failed:
>  - src is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'src'

Error Recognising Face: OpenCV(4.8.0) :-1: error: (-5:Bad argument) in function 'cvtColor'
> Overload resolution failed:
>  - src is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'src'

Error Recognising Face: OpenCV(4.8.0) :-1: error: (-5:Bad argument) in function 'cvtColor'
> Overload resolution failed:
>  - src is

(66, 33.772552870952886)
(66, 32.855155500581496)
(66, 36.208420171476405)
(64, 37.351980308486375)
(64, 37.351980308486375)
(19, 28.998659124198124)
(14, 36.95485288048104)
(64, 52.8839770007)
(64, 52.8839770007)
(66, 34.04613418619472)
(66, 34.04613418619472)
(64, 33.39977639140665)
(71, 36.28011719249774)
(71, 36.28011719249774)
(68, 28.533526237869673)
(73, 31.19280747512212)
(73, 31.19280747512212)
(66, 29.177117100158146)
(71, 28.519964584784407)
(71, 28.519964584784407)
(71, 28.519964584784407)
(86, 43.084214865684885)
(77, 39.02802438328716)
(77, 39.02802438328716)
(71, 34.73121786988595)
(71, 34.73121786988595)
(63, 33.78031311669928)
(44, 32.14552886465039)
(44, 32.14552886465039)
(44, 32.14552886465039)
(58, 32.07244811238726)
(58, 32.07244811238726)
(50, 31.55545058821804)
(60, 31.1870950307193)
(60, 31.1870950307193)
(60, 31.1870950307193)
(47, 32.18549780538965)
(47, 32.18549780538965)
(81, 32.31029057437142)
(64, 33.539509443469775)
(64, 33.539509443469775)
Error Recogni

(2, 37.270186427429245)
(2, 37.270186427429245)
(43, 33.240594323854175)
(58, 34.59416205186243)
(43, 33.511548014133155)
(43, 32.63807507633606)
(43, 32.63807507633606)
(43, 32.63807507633606)
(51, 36.882121383427005)
(57, 32.13923607660213)
(57, 32.13923607660213)
(93, 31.300340253498806)
(93, 31.300340253498806)
(58, 33.97205361399581)
(58, 33.767071618451475)
(58, 33.767071618451475)
(93, 32.945844062634755)
(2, 31.87475580933397)
(2, 31.87475580933397)
Error Recognising Face: OpenCV(4.8.0) :-1: error: (-5:Bad argument) in function 'cvtColor'
> Overload resolution failed:
>  - src is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'src'

Error Recognising Face: OpenCV(4.8.0) :-1: error: (-5:Bad argument) in function 'cvtColor'
> Overload resolution failed:
>  - src is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'src'

Error Recognising Face: OpenCV(4.8.0) :-1: error: (-5:Bad argument) in function 'cvtColor'
> Overload re

(57, 59.31310069922796)
(57, 59.31310069922796)
(92, 62.52827880878694)
(92, 62.52827880878694)
(57, 57.94979365348126)
(44, 35.2758522721442)
(44, 35.2758522721442)
(44, 35.2758522721442)
(50, 36.43810238620203)
(50, 36.43810238620203)
(66, 38.354869949982735)
(63, 34.182560860966525)
(63, 34.182560860966525)
(63, 34.182560860966525)
(66, 34.320661805564676)
(66, 34.320661805564676)
(66, 38.22507439441896)
(81, 38.990294205296834)
(81, 38.990294205296834)
(81, 38.990294205296834)
(66, 37.55733213352435)
(66, 37.55733213352435)
(66, 37.611031896509296)
(92, 38.3078594413141)
(92, 38.3078594413141)
(92, 38.3078594413141)
(66, 39.22331127049599)
(66, 39.22331127049599)
(81, 37.195409679079525)
(44, 36.22924441839508)
(44, 36.22924441839508)
(2, 39.22600034951439)
(43, 37.82392614612043)
(43, 37.82392614612043)
(43, 37.82392614612043)
(37, 33.02723246642307)
(37, 33.02723246642307)
(37, 37.047278569067444)
(37, 38.066149988951956)
(37, 38.066149988951956)
(37, 38.066149988951956)
(53, 35.